# Tutorial from
https://www.youtube.com/watch?v=BQ4kVn-Rt84

In [1]:
import os
import torch
import numpy as np
import plotly

In [2]:
def target_function_0(individuals):
    result = []
    for x in individuals:
        result.append(
            np.exp(-((x[0] - 2) ** 2))
            + np.exp(-((x[0] - 6) ** 2) / 10)
            + 1 / (x[0] ** 2 + 1)
        )
    return torch.tensor(result)

def target_function_1(individuals):
    result = []
    for x in individuals:
        result.append(
            np.sin(-((x[0] - 2) ** 2)) + (-((x[0] - 6) ** 2) / 10) + 1 / (x[0] ** 2 + 1)
        )
    return torch.tensor(result)


In [3]:
import plotly.graph_objects as go

def plot_target_functions():
    x = np.linspace(-2., 10., 100)
    x_new = x.reshape((100, -1))
    z0 = target_function_0(x_new)
    z1 = target_function_1(x_new)

    data0 = go.Scatter(x=x, y=z0, line_color="#FE73FF")
    data1 = go.Scatter(x=x, y=z1, line_color="#FE73FF")

    fig = go.Figure(data=data0)
    fig.update_layout(title="target function 0", xaxis_title="x", yaxis_title="y")
    fig.show()

    fig = go.Figure(data=data1)
    fig.update_layout(title="target function 1", xaxis_title="x", yaxis_title="y")
    fig.show()

plot_target_functions()

In [4]:
def define_playground_data_and_objectives(verbose=False, data_size=40):
    X1 = torch.rand(data_size, 1)
    X2 = X1
    i1, i2 = torch.zeros(data_size, 1), torch.ones(data_size, 1)
    # x1 and X2 are the same data points - but still two target functions
    train_x = torch.cat([torch.cat([X1, i1], -1), torch.cat([X2, i2], -1)])
    exact_objs = torch.cat(
        (target_function_0(X1), target_function_1(X2)), dim=-1
    ).unsqueeze(
        -1
    )  # previousversion
    best_observed_values = torch.tensor(
        [target_function_0(X1).max().item(), target_function_1(X2).max().item()]
    ).unsqueeze(-1)
    # best_observed_value_mean = best_observed_values.mean()
    # just the max on the second target function
    the_best_observed_value = torch.tensor(target_function_1(X2).max().item())

    if verbose:
        print(train_x.shape)
        print(exact_objs.shape)
        print(exact_objs)
        print(best_observed_values.shape)
        # print(best_observed_value_mean.shape)
        print(the_best_observed_value.shape)

    return train_x, exact_objs, the_best_observed_value

In [5]:

# model
from botorch.models import SingleTaskGP, MultiTaskGP, FixedNoiseMultiTaskGP
from botorch.acquisition.objective import IdentityMCObjective, LinearMCObjective
from gpytorch.mlls import ExactMarginalLogLikelihood 
from gpytorch.likelihoods import MultitaskGaussianLikelihood
# fitting model
from botorch import fit_gpytorch_model , fit_gpytorch_mll
# acquisition function
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.optim import optimize_acqf

init_x, init_y, best_init_y = define_playground_data_and_objectives(verbose=True)

# define bounds for data to be sampled
bounds = torch.tensor([[0.],[10.]])

# model
multi_model = MultiTaskGP(
    init_x,
    init_y, 
    task_feature=-1, 
)

mll = ExactMarginalLogLikelihood(multi_model.likelihood, multi_model)

# fit the model
fit_gpytorch_mll(mll)

# Objective
objective = LinearMCObjective(torch.tensor([0., 1.]))
# exact_objs
# objective                    
# acquisition function
ei = qExpectedImprovement(
    model=multi_model, 
    best_f=best_init_y,
    objective=objective,
)

# pick candidates
n_points=10
candidates, _ = optimize_acqf(
    acq_function=ei, 
    bounds=bounds,
    q=n_points,
    num_restarts=200,
    raw_samples=512,
    # options={'batch_limit': 5, 'maxiter': 200},
)


torch.Size([80, 2])
torch.Size([80, 1])
tensor([[ 0.9193],
        [ 0.9244],
        [ 0.9254],
        [ 0.9668],
        [ 0.9427],
        [ 0.9132],
        [ 0.9660],
        [ 1.0420],
        [ 0.9978],
        [ 0.9644],
        [ 1.0369],
        [ 0.9253],
        [ 0.9200],
        [ 0.9130],
        [ 1.0168],
        [ 0.9492],
        [ 0.9270],
        [ 1.0463],
        [ 0.9334],
        [ 1.0309],
        [ 0.9130],
        [ 0.9161],
        [ 0.9158],
        [ 0.9175],
        [ 0.9130],
        [ 0.9363],
        [ 0.9700],
        [ 0.9147],
        [ 0.9193],
        [ 1.0353],
        [ 0.9610],
        [ 0.9192],
        [ 1.0322],
        [ 1.0225],
        [ 0.9462],
        [ 0.9183],
        [ 1.0347],
        [ 0.9191],
        [ 0.9237],
        [ 0.9156],
        [-3.0254],
        [-2.9876],
        [-3.1201],
        [-2.9300],
        [-3.0579],
        [-3.1173],
        [-2.9349],
        [-2.1917],
        [-2.7068],
        [-2.9444],
        [-

/Users/jean/opt/anaconda3/envs/pe/lib/python3.11/site-packages/botorch/models/gpytorch.py:129: UserWarning:

The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444



In [6]:
candidates

tensor([[ 5.5207],
        [ 0.0000],
        [ 8.2142],
        [ 6.3420],
        [ 9.0787],
        [10.0000],
        [ 7.0939],
        [ 2.5549],
        [ 4.4834],
        [ 3.4440]])

In [7]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [8]:
n_runs = 10
init_x, init_y, best_init_y = generate_initial_data(20)
bounds = torch.tensor([[0.],[10.]])

for i in range(n_runs):
    print(f"Nr. of optimization run: {i}")
    new_candidates = get_next_points(init_x, init_y, best_init_y, bounds, n_points=6)
    new_results = target_function(new_candidates).unsqueeze(-1)
    
    print(f"New candidates are: {new_candidates}")
    init_x = torch.cat([init_x, new_candidates])
    init_y = torch.cat([init_y, new_results])
    
    best_init_y = init_y.max().item()
    print(f"Best point performs this ways: {best_init_y}")

NameError: name 'generate_initial_data' is not defined

our options:
- For the report we do the report for all the different tasks. \
1/ For the selection we can do a single task module on the summary variable - then we can do the pred on the mutivariate model to egt the individual predictions \
2/ We manage to do botorch on the multivariate thing 

What I need to do:
- make this work with a multivariate botorch model

    - generate multitarget + summary_var
    - get candidates - (10000) 
    - do the BO on summary_var --> get suggestions (batch of 96)
    - candidates -> predict the different targets (report purpose)

- make it work with our model
